# New Dataset Analysis

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#To display all columns
pd.options.display.max_columns = None
pd.options.display.max_rows = None

## Utility Functions

#### Histogram

In [3]:
# Create a histogram
def seaHistogram(dataframe_to_plot, column_of_interest, figsize=(8,10), title=None, xlabel=None, 
                  ylabel=None, hue=None, weights=None, palette="deep", stat="count", binwidth=None,
                  multiple="layer", element="step", fill=True, shrink=1, kde=False, color="blue", 
                  log_scale=None, ylog=False, xlog=False, show_grid=False, vline=False,
                  n_vline=0, c_vline="red", discrete=False, kde_kws=None):
    """
    This function creates a histogram using seaborn library with customizable options.

    Parameters:
    - dataframe_to_plot (DataFrame): DataFrame containing the data to be plotted.
    - column_of_interest (list or str): Name of the column to be plotted on the x-axis.
    - figsize (tuple, optional): Size of the figure (width, height).
    - title (str, optional): Title of the plot.
    - xlabel (str, optional): Label for the x-axis.
    - ylabel (str, optional): Label for the y-axis.
    - hue (str, optional): Variable used for grouping the data.
    - weights (str, optional): Weight the contribution of the corresponding data points.
    - palette (str, list or dict, optional): Color palette for the plot.
    - stat (str, , optional): Statistic to compute within each bin (count, density, probability).
    - binwidth (number, optional): Width of each bin.
    - multiple (str, optional): Method for plotting multiple histograms (layer, dodge, stack).
    - element (str, optional): Element to draw (step, poly).
    - fill (bool, optional): Whether to fill the histogram bars.
    - shrink (number, optional): Scaling factor for the width of the step/poly elements.
    - kde (bool, optional): Whether to plot a kernel density estimate.
    - color (str, optional): Color of the histogram bars (if hue is not specified).
    - log_scale (str, optional): Scaling of the x or y-axis (linear, log, symlog, logit).
    - ylog (bool, optional): Whether to use a logarithmic scale for the y-axis.
    - xlog (bool, optional): Whether to use a logarithmic scale for the x-axis.
    - show_grid (bool, optional): Whether to show grid lines on the plot.
    - vline (bool, optional): Add a vertical red line.
    - n_vline (number, optional): Position in x-axis of the vline.
    - c_vline (str, optional): Color of the vline.
    - discrete (bool, optional): If True, default to binwidth=1.
    - kde_kws (dict, optional): Parameters that control the KDE computation.


    Returns:
    - None: The function displays the plot.
    """

    plt.figure(figsize=figsize)
    
    if hue:
        sns.histplot(data=dataframe_to_plot, x=column_of_interest, hue=hue, weights=weights, stat=stat, 
                     binwidth=binwidth, multiple=multiple, element=element, fill=fill, 
                     shrink=shrink, kde=kde, palette=palette, log_scale=log_scale, discrete=discrete,
                     kde_kws=kde_kws)
    else:
        sns.histplot(data=dataframe_to_plot, x=column_of_interest, weights=weights, stat=stat, 
                     binwidth=binwidth, element=element, fill=fill,
                     shrink=shrink, kde=kde, color=color, log_scale=log_scale, discrete=discrete,
                     kde_kws=kde_kws)
        
    if ylog:
        plt.yscale("log")

    if xlog:
        plt.xscale("log")
    
    if vline:
        # Add a vertical red
        plt.axvline(x=n_vline, color=c_vline, linestyle='--')  

    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)

    if show_grid:
        plt.grid(True)

    plt.show()

#### Null Values Percentage

In [4]:
def printPercentageNullValues(dataframe, perc=50):
    '''
    This function prints the columns of a DataFrame that have more than 
    perc percent of null values. 
    Input:
    - dataframe: pandas DataFrame to be analysed
    - perc: percentage threshold (default 50) to consider a column 
            as containing a high percentage of null values
    '''
    
    # Number of rows of dataframe
    total_rows = len(dataframe)
    
    missing_data = dataframe.isna().sum()
    missing_percentage = (missing_data / total_rows) * 100
    
    # Filter columns with more than perc of null values
    high_missing_cols = missing_percentage[missing_percentage > perc]
    high_missing_cols = high_missing_cols.sort_values(ascending=False)
    
    # Print the columns 
    if not high_missing_cols.empty:
        print("Columns with more than",  perc, "% of null values")
        print(high_missing_cols)
    else:
        print("No column has more than", perc, "% of null values.")

#### Sorting Binary Columns

In [5]:
def sortingBinaryColumns(dataframe, columns):
    """
    Reorders the columns of the DataFrame based on the number of unique values in each column.
    Columns with only two unique values are placed first, followed by other columns.

    Parameters:
    - dataframe (DataFrame): The pandas DataFrame.
    - columns (list): A list containing the names of the columns to be reordered.

    Returns:
    - list: A list containing the reordered column names.
    """

    reordered_columns = []  # List to store column names with only two unique values
    non_bin_columns = []     # List to store column names with more than two unique values

    # Iterate through the specified columns
    for col in columns:
        # Check if the number of unique values in the column is equal to 2
        if dataframe[col].nunique() == 2:
            # If yes, append the column name to reordered_columns
            reordered_columns.append(col)
        else:
            # If no, append the column name to non_bin_columns
            non_bin_columns.append(col)

    # Combine the two lists to get the final reordered column order
    reordered_columns = reordered_columns + non_bin_columns
    
    return reordered_columns

## Loading Dataset

Let's start by loading the dataset into the `newData` dataframe and analyzing its key characteristics to better understand the data we're working with.

In [6]:
newData=pd.read_excel("Datasets/NewDataset/URRAH_TG.xlsx")

In [7]:
newData.head(10)

,SESSO,SESSO0,ETA,FAM_IPERT,FAM_CV,DIABETE,FUMO_ATT,EXFUMO,ALCOOL,ETANOLO_G_DIE,ATT_FIS,ATT_FIS_H_SETT,SAPEVA,IRC,GOTTA,ALLOPURINOLO,PESO,ALTEZZA,BMI,VITA,PAS,PAD,FC,CREATININA,AZOTEMIA,URICEMIA,ALB_CAT,ALB_CREAT_MG_MMOL_3_4_34_MICRO,ALB_CREAT_MG_G_30_300_MICRO,ALBURIA_MG_24H_30_300,ALBURIA_MG_DL,NORMOMICROMACRO,GLICEMIA,CT,HDL,TG,HB,HT,LVH,IMT,PLACCA,INS_CARD,ACEI,SARTANI,CAA_DIID,ALTRO_CAA,BETABLOCCANTI,DIURETICI,HCTZ,INDAPAMIDE,CLORTALIDONE,DELL_ANSA,STATINA,STATO_AL_FU,NF_IMA,F_IMA,NF_CBV,F_CBV,NF_HF,F_HF,RIV_COR,MORTE_CV,IPERTESO,FU,FU_NF_IMA,FU_F_IMA,FU_NF_CBV,FU_F_CBV,FU_NF_HF,FU_F_HF,PCR,VES,OMEGA_3,FIBRATI,EZETEMIBE,ACIDO_NICOTINICO,RESINE,MENOPAUSA,ANNI_MENOP,IMA_BASE,CBV_BASE,FA_BASE,HF_BASE,NF_FA,FU_NF_FA
0,1,UOMO,70.0,0.0,1.0,0.0,1.0,0.0,1.0,32.00000,0.0,NaN,0.0,0.0,0.0,NaN,NaN,167.0,NaN,86.0,140.0,98.0,51.0,1.1,29.6,5.4,0.0,0.2369,2.0942,NaN,NaN,0.0,88.0,254.0,55.0,130.0,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,192.000000,192.0,192.0,192.0,192.0,192.0,192.0,2.11,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0
1,0,DONNA,23.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,NaN,0.0,0.0,0.0,NaN,NaN,169.0,NaN,77.0,114.0,70.0,79.0,0.9,23.6,3.5,0.0,0.2173,1.9212,NaN,NaN,0.0,82.0,178.0,56.0,66.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,192.000000,192.0,192.0,192.0,192.0,192.0,192.0,0.91,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0
2,0,DONNA,78.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,1.0,1.0,0.0,NaN,NaN,155.0,NaN,107.0,163.0,84.0,77.0,1.0,26.6,5.9,0.0,0.2772,2.4503,NaN,NaN,0.0,101.0,271.0,74.0,133.0,NaN,NaN,NaN,NaN,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,192.000000,192.0,192.0,192.0,192.0,192.0,192.0,0.28,NaN,0.0,0.0,0.0,0.0,0.0,1.0,30.25,0.0,0.0,0.0,0.0,0.0,0.0
3,0,DONNA,53.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,143.0,81.0,65.0,0.9,23.6,3.0,0.0,0.3835,3.3898,NaN,NaN,0.0,85.0,202.0,41.0,149.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,192.000000,192.0,192.0,192.0,192.0,192.0,192.0,1.31,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0
4,1,UOMO,82.0,1.0,0.0,0.0,0.0,0.0,1.0,32.00000,0.0,NaN,0.0,1.0,0.0,NaN,NaN,155.0,NaN,78.0,147.0,59.0,49.0,1.2,32.6,4.0,0.0,1.5496,13.6986,NaN,NaN,0.0,107.0,209.0,69.0,97.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,192.000000,192.0,192.0,192.0,192.0,192.0,192.0,0.33,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0
5,0,DONNA,67.0,1.0,0.0,1.0,0.0,0.0,0.0,NaN,0.0,NaN,1.0,1.0,0.0,NaN,NaN,156.0,NaN,98.0,150.0,76.0,61.0,1.3,35.6,4.4,0.0,0.6674,5.8996,NaN,NaN,0.0,234.0,223.0,41.0,256.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,192.000000,192.0,192.0,192.0,192.0,192.0,192.0,2.25,NaN,0.0,0.0,0.0,0.0,0.0,1.0,14.62,0.0,0.0,0.0,0.0,0.0,0.0
6,0,DONNA,49.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,1.0,0.0,0.0,NaN,NaN,164.0,NaN,89.0,130.0,85.0,60.0,0.8,20.6,3.6,0.0,0.1590,1.4055,NaN,NaN,0.0,89.0,182.0,68.0,64.0,NaN,NaN,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,192.000000,192.0,192.0,192.0,192.0,192.0,192.0,0.22,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0
7,1,UOMO,65.0,NaN,NaN,0.0,0.0,1.0,1.0,12.34375,NaN,NaN,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,103.0,63.0,80.0,1.1,23.0,5.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,144.0,27.0,68.0,13.3,42.700001,NaN,NaN,NaN,1.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0,26.800000,26.0,26.0,26.0,26.0,26.0,26.0,NaN,6.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
8,0,DONNA,87.0,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,1.0,0.0,NaN,0.0,54.2,NaN,NaN,107.0,158.0,83.0,87.0,1.4,50.0,4.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,173.0,64.0,109.0,9.6,30.600000,NaN,1.1,1.0,1.0,0.0,0.0,1.0,NaN,0.0,0.0,0.0,0.0,

To understand the distribution of our dataset, we utilize the `.info()` and `.describe()` commands from pandas. The focus is on all numerical columns, excluding the `SESSO0` column, the unique `object` column. Here's a breakdown based on the `.describe()` output:

1. **Data Completeness**: Many columns have different counts, suggesting varying levels of missing data across the dataset. Some variables have nearly complete data (e.g., "ETA" with 27,074 entries), while others have considerably fewer observations (e.g., "VES" with only 680 entries). 

2. **Distributions**: 
   - Many of the columns seem to contain binary data.
   - Variables like "BMI", "PESO", and "ALTEZZA" have continuous distributions, with reasonably spread out quartiles and larger standard deviations, indicating more variability in these health-related metrics.

3. **Central Tendency**:
   - For many columns, the mean and median (50th percentile) are close, indicating a relatively symmetric distribution. However, this may not be the case for some variables where the standard deviation is large relative to the mean, suggesting skewed distributions or the presence of outliers.
   - For health indicators like "BMI" and blood pressure ("PAS", "PAD"), the mean and median values fall within expected ranges for adult populations, though some outliers or extreme values may exist (as indicated by the minimum and maximum values).

In [8]:
newData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27078 entries, 0 to 27077
Data columns (total 85 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   SESSO                           27078 non-null  int64  
 1   SESSO0                          27078 non-null  object 
 2   ETA                             27074 non-null  float64
 3   FAM_IPERT                       18938 non-null  float64
 4   FAM_CV                          16383 non-null  float64
 5   DIABETE                         25902 non-null  float64
 6   FUMO_ATT                        24526 non-null  float64
 7   EXFUMO                          22055 non-null  float64
 8   ALCOOL                          19514 non-null  float64
 9   ETANOLO_G_DIE                   7230 non-null   float64
 10  ATT_FIS                         13605 non-null  float64
 11  ATT_FIS_H_SETT                  6797 non-null   float64
 12  SAPEVA                          

In [9]:
newData.describe()

,SESSO,ETA,FAM_IPERT,FAM_CV,DIABETE,FUMO_ATT,EXFUMO,ALCOOL,ETANOLO_G_DIE,ATT_FIS,ATT_FIS_H_SETT,SAPEVA,IRC,GOTTA,ALLOPURINOLO,PESO,ALTEZZA,BMI,VITA,PAS,PAD,FC,CREATININA,AZOTEMIA,URICEMIA,ALB_CAT,ALB_CREAT_MG_MMOL_3_4_34_MICRO,ALB_CREAT_MG_G_30_300_MICRO,ALBURIA_MG_24H_30_300,ALBURIA_MG_DL,NORMOMICROMACRO,GLICEMIA,CT,HDL,TG,HB,HT,LVH,IMT,PLACCA,INS_CARD,ACEI,SARTANI,CAA_DIID,ALTRO_CAA,BETABLOCCANTI,DIURETICI,HCTZ,INDAPAMIDE,CLORTALIDONE,DELL_ANSA,STATINA,STATO_AL_FU,NF_IMA,F_IMA,NF_CBV,F_CBV,NF_HF,F_HF,RIV_COR,MORTE_CV,IPERTESO,FU,FU_NF_IMA,FU_F_IMA,FU_NF_CBV,FU_F_CBV,FU_NF_HF,FU_F_HF,PCR,VES,OMEGA_3,FIBRATI,EZETEMIBE,ACIDO_NICOTINICO,RESINE,MENOPAUSA,ANNI_MENOP,IMA_BASE,CBV_BASE,FA_BASE,HF_BASE,NF_FA,FU_NF_FA
count,27078.000000,27074.000000,18938.000000,16383.000000,25902.000000,24526.000000,22055.000000,19514.000000,7230.000000,13605.000000,6797.000000,19448.000000,25892.000000,14050.000000,16049.000000,24528.000000,24494.000000,24454.000000,12869.000000,26489.000000,26494.000000,22432.000000,24705.000000,14153.000000,27078.000000,16909.000000,2398.000000,2708.000000,1905.000000,1279.000000,6677.000000,22009.000000,26041.000000,22936.000000,27078.000000,18561.000000,20495.000000,171.000000,3921.000000,10936.000000,20233.000000,20352.000000,17277.000000,24637.000000,13853.000000,23562.000000,23712.000000,9741.000000,9729.000000,12937.000000,15867.000000,25734.000000,21247.000000,20763.000000,19932.000000,20748.000000,19881.000000,19687.000000,16933.000000,12592.000000,21938.000000,27078.000000,22833.000000,22265.000000,22266.000000,22269.000000,22266.000000,22264.000000,22265.000000,3844.000000,680.000000,4005.000000,4696.000000,4439.000000,3394.0,3394.000000,3170.000000,1447.000000,1576.0,1576.0,3612.000000,1576.0,2283.000000,2275.000000
mean,0.500000,56.524057,0.535590,0.384362,0.112771,0.257074,0.207073,0.602183,27.571404,0.534436,14.296431,0.440817,0.166036,0.017580,0.025609,72.332422,164.655132,26.634106,90.842567,142.173320,84.728647,71.595768,0.958226,36.176164,5.058203,0.266308,1.020581,38.270789,141.877496,1.541248,0.196645,98.360187,209.662784,52.883754,126.727608,14.271313,42.370152,0.280702,0.974241,0.229151,0.115603,0.164406,0.120507,0.102813,0.013499,0.093456,0.171010,0.093625,0.012540,0.017237,0.061826,0.068159,0.148586,0.021962,0.021674,0.019038,0.018259,0.013816,0.026162,0.013739,0.070745,0.660795,130.333676,130.165776,115.617219,129.148727,129.671248,115.546128,130.121581,2.360973,19.391176,0.055680,0.008944,0.023879,0.0,0.001473,0.460883,16.831735,0.0,0.0,0.031285,0.0,0.019273,0.050110
std,0.500009,15.253567,0.498745,0.486459,0.316319,0.437030,0.405218,0.489460,28.839968,0.498831,21.245328,0.496498,0.372120,0.131424,0.157971,13.909713,9.747290,4.330725,12.777931,22.812086,12.535061,12.308247,0.656884,15.733269,1.422008,0.442040,5.151723,340.693513,463.135863,3.927762,0.470967,24.686763,40.056957,14.849407,76.760740,1.437432,3.930261,0.450662,0.517665,0.420306,0.319756,0.370653,0.325563,0.303720,0.115402,0.291076,0.376526,0.291321,0.111283,0.130160,0.240848,0.252023,0.355688,0.146563,0.145619,0.136662,0.133889,0.116731,0.235294,0.116410,0.256404,0.473448,68.278615,67.965358,75.144535,68.348122,68.180031,75.132047,68.173818,13.036118,15.962029,0.229332,0.094158,0.152690,0.0,0.038360,0.498546,12.140815,0.0,0.0,0.174110,0.0,0.137513,0.481992
min,0.000000,18.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,31.799999,129.000000,13.619612,42.000000,70.000000,39.000000,30.000000,0.100000,0.180000,1.000000,0.000000,0.058200,0.008000,0.000000,0.039600,0.000000,40.000000,54.000000,10.000000,20.000000,5.250000,15.300000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,2.000000,0.000000,0.000000,0.00000

Let us now analyze the number of **null values** for each column. Specifically, we will print in descending order the columns that have a null value count greater than 35% of the total number of rows in the dataset. Next to each column name, we will also report the percentage of null values. We will use the function `printPercentageNullValues`, defined in the utility functions.

In [10]:
printPercentageNullValues(newData, 35)

Columns with more than 35 % of null values
LVH                               99.368491
VES                               97.488736
ALBURIA_MG_DL                     95.276608
ANNI_MENOP                        94.656178
CBV_BASE                          94.179777
IMA_BASE                          94.179777
HF_BASE                           94.179777
ALBURIA_MG_24H_30_300             92.964768
FU_NF_FA                          91.598346
NF_FA                             91.568801
ALB_CREAT_MG_MMOL_3_4_34_MICRO    91.144102
ALB_CREAT_MG_G_30_300_MICRO       89.999261
MENOPAUSA                         88.293079
ACIDO_NICOTINICO                  87.465839
RESINE                            87.465839
FA_BASE                           86.660758
PCR                               85.803974
IMT                               85.519610
OMEGA_3                           85.209395
EZETEMIBE                         83.606618
FIBRATI                           82.657508
NORMOMICROMACRO                  

## Data Cleaning

Having briefly studied what data we are going to work with, let us start cleaning them. We first remove any duplicates, using the `drop_duplicates()` function.

In [11]:
N=len(newData)
newData=newData.drop_duplicates()
numberOfDuplicates=N-len(newData)
print("The number of duplicates in the DataFrame was:", numberOfDuplicates)

The number of duplicates in the DataFrame was: 105


Upon examining the columns `SESSO0` and `SESSO`, we observe that they contain the same data but are formatted differently. Therefore, we will eliminate `SESSO0` and retain `SESSO`. However, we will rename `SESSO` to reflect the information derived from `SESSO0`, where 1 represents **Male** and 0 represents **Female**.

In [12]:
newData=newData.drop(columns="SESSO0")
newData=newData.rename(columns={"SESSO": "Gender (Male = 1)"})

### Number of Values in the Columns

In this section, we will analyze which columns are **binary** and which are **continuous**. Before we begin this analysis, we want to check if there are any columns containing fewer than two distinct values.

In [13]:
columnsWithSingleValue=[]

for col in newData.columns:
    if newData[col].nunique() < 2:
        
        columnsWithSingleValue.append(col)
        
        print("Value Counts:")
        print(newData[col].value_counts())
        print("----------------")
        print("Null Values Percentage:")
        print((newData[col].isna().sum()/ len(newData)) * 100)
        print("")

Value Counts:
ACIDO_NICOTINICO
0.0    3394
Name: count, dtype: int64
----------------
Null Values Percentage:
87.41704667630593

Value Counts:
IMA_BASE
0.0    1576
Name: count, dtype: int64
----------------
Null Values Percentage:
94.15712008304601

Value Counts:
CBV_BASE
0.0    1576
Name: count, dtype: int64
----------------
Null Values Percentage:
94.15712008304601

Value Counts:
HF_BASE
0.0    1576
Name: count, dtype: int64
----------------
Null Values Percentage:
94.15712008304601



There are four columns that contain only a single value, and the number of instances of this value is very low. In fact, they all have extremely high percentages of null values. We consider these columns invalid and will remove them from the dataset.

In [14]:
newData=newData.drop(columns=columnsWithSingleValue)

Now we count how many binary columns are in our dataframe. Then we reorder the columns of `newData` so that **binary columns** come first, followed by **continuous columns()**. To achieve this, we use the `reorder_columns` utility function defined in the previous section.

In [15]:
numberBinaryColumns=sum(newData[col].nunique() == 2 for col in newData.columns)

print("The number of binary column is:", numberBinaryColumns)
print("The number of continous columns is:", newData.shape[1]-numberBinaryColumns)

The number of binary column is: 43
The number of continous columns is: 37


In [16]:
sortingColumns=sortingBinaryColumns(newData, newData.columns)
newData = newData.reindex(columns=sortingColumns)

newData.head()

,Gender (Male = 1),FAM_IPERT,FAM_CV,DIABETE,FUMO_ATT,EXFUMO,ALCOOL,ATT_FIS,SAPEVA,IRC,GOTTA,ALLOPURINOLO,ALB_CAT,LVH,PLACCA,INS_CARD,ACEI,SARTANI,CAA_DIID,ALTRO_CAA,BETABLOCCANTI,DIURETICI,HCTZ,INDAPAMIDE,CLORTALIDONE,DELL_ANSA,STATINA,STATO_AL_FU,NF_IMA,F_IMA,NF_CBV,F_CBV,NF_HF,RIV_COR,MORTE_CV,IPERTESO,OMEGA_3,FIBRATI,EZETEMIBE,RESINE,MENOPAUSA,FA_BASE,NF_FA,ETA,ETANOLO_G_DIE,ATT_FIS_H_SETT,PESO,ALTEZZA,BMI,VITA,PAS,PAD,FC,CREATININA,AZOTEMIA,URICEMIA,ALB_CREAT_MG_MMOL_3_4_34_MICRO,ALB_CREAT_MG_G_30_300_MICRO,ALBURIA_MG_24H_30_300,ALBURIA_MG_DL,NORMOMICROMACRO,GLICEMIA,CT,HDL,TG,HB,HT,IMT,F_HF,FU,FU_NF_IMA,FU_F_IMA,FU_NF_CBV,FU_F_CBV,FU_NF_HF,FU_F_HF,PCR,VES,ANNI_MENOP,FU_NF_FA
0,1,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,70.0,32.0,NaN,NaN,167.0,NaN,86.0,140.0,98.0,51.0,1.1,29.6,5.4,0.2369,2.0942,NaN,NaN,0.0,88.0,254.0,55.0,130.0,NaN,NaN,NaN,0.0,192.0,192.0,192.0,192.0,192.0,192.0,192.0,2.11,NaN,NaN,0.0
1,0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.0,NaN,NaN,NaN,169.0,NaN,77.0,114.0,70.0,79.0,0.9,23.6,3.5,0.2173,1.9212,NaN,NaN,0.0,82.0,178.0,56.0,66.0,NaN,NaN,NaN,0.0,192.0,192.0,192.0,192.0,192.0,192.0,192.0,0.91,NaN,NaN,0.0
2,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,NaN,0.0,NaN,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,78.0,NaN,NaN,NaN,155.0,NaN,107.0,163.0,84.0,77.0,1.0,26.6,5.9,0.2772,2.4503,NaN,NaN,0.0,101.0,271.0,74.0,133.0,NaN,NaN,NaN,0.0,192.0,192.0,192.0,192.0,192.0,192.0,192.0,0.28,NaN,30.25,0.0
3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,53.0,NaN,NaN,NaN,NaN,NaN,NaN,143.0,81.0,65.0,0.9,23.6,3.0,0.3835,3.3898,NaN,NaN,0.0,85.0,202.0,41.0,149.0,NaN,NaN,NaN,0.0,192.0,192.0,192.0,192.0,192.0,192.0,192.0,1.31,NaN,NaN,0.0
4,1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,82.0,32.0,NaN,NaN,155.0,NaN,78.0,147.0,59.0,49.0,1.2,32.6,4.0,1.5496,13.6986,NaN,NaN,0.0,107.0,209.0,69.0,97.0,NaN,NaN,NaN,0.0,192.0,192.0,192.0,192.0,192.0,192.0,192.0,0.33,NaN,NaN,0.0


### Similarity with Original Data

The current `newData` we are working with is intended to extend the **original dataset** we previously analyzed. After examining both datasets, we identified several **common columns**, albeit with different names. To facilitate future operations, we will rename the columns in this dataset, using `.rename()` command, to match the names used in the original dataset. This will enable easier merging of the datasets in the future by using the common columns as a reference.

In [17]:
columnNameMapping = {
    'ETA': 'Age',
    'FAM_IPERT': 'History of hypertension',
    'FUMO_ATT': 'Smoke',
    'EXFUMO': 'History of smoke',
    'DIABETE': 'Diabetes',
    'PESO': 'Weight',
    'ALTEZZA': 'Height',
    'BMI': 'BMI',
    'PAS': 'Systolic blood pressure',
    'PAD': 'Diastolic blood pressure',
    'CREATININA': 'Creatinina',
    'GLICEMIA': 'Glycemia',
    'CT': 'Total cholesterol',
    'HDL': 'HDL',
    'TG': 'Triglycerides',
    'INS_CARD': 'Coronary Artery Disease (CAD)',
    'BETABLOCCANTI': 'B-Blockers',
    'ACEI': 'ACE-inhibitors',
    'DIURETICI': 'Diuretics',
    'MORTE_CV': 'CV Death',
}

In [18]:
newData=newData.rename(columns=columnNameMapping)

At this point, let's load the old data into the `oldData` dataframe and analyze the total number of columns in both datasets. We will also determine how many of these columns are common between the two datasets.

In [19]:
oldData=pd.read_csv("Datasets/Cleaned_Dataset_7Y/chl_dataset_known.csv")

In [20]:
print("Number of columns of newData:", newData.shape[1])
print("Number of columns of oldData:", oldData.shape[1])
print("--------------------------------")
print("Number of common columns:", len(columnNameMapping)+1)

Number of columns of newData: 80
Number of columns of oldData: 69
--------------------------------
Number of common columns: 21


At this point, let's also print the names of the columns from each of the two datasets.

In [21]:
print("newData columns name:")
print(newData.columns)

newData columns name:
Index(['Gender (Male = 1)', 'History of hypertension', 'FAM_CV', 'Diabetes',
       'Smoke', 'History of smoke', 'ALCOOL', 'ATT_FIS', 'SAPEVA', 'IRC',
       'GOTTA', 'ALLOPURINOLO', 'ALB_CAT', 'LVH', 'PLACCA',
       'Coronary Artery Disease (CAD)', 'ACE-inhibitors', 'SARTANI',
       'CAA_DIID', 'ALTRO_CAA', 'B-Blockers', 'Diuretics', 'HCTZ',
       'INDAPAMIDE', 'CLORTALIDONE', 'DELL_ANSA', 'STATINA', 'STATO_AL_FU',
       'NF_IMA', 'F_IMA', 'NF_CBV', 'F_CBV', 'NF_HF', 'RIV_COR', 'CV Death',
       'IPERTESO', 'OMEGA_3', 'FIBRATI', 'EZETEMIBE', 'RESINE', 'MENOPAUSA',
       'FA_BASE', 'NF_FA', 'Age', 'ETANOLO_G_DIE', 'ATT_FIS_H_SETT', 'Weight',
       'Height', 'BMI', 'VITA', 'Systolic blood pressure',
       'Diastolic blood pressure', 'FC', 'Creatinina', 'AZOTEMIA', 'URICEMIA',
       'ALB_CREAT_MG_MMOL_3_4_34_MICRO', 'ALB_CREAT_MG_G_30_300_MICRO',
       'ALBURIA_MG_24H_30_300', 'ALBURIA_MG_DL', 'NORMOMICROMACRO', 'Glycemia',
       'Total cholesterol', 'HDL

In [22]:
print("oldData colums name:")
print(oldData.columns)

oldData colums name:
Index(['Gender (Male = 1)', 'Euthyroid',
       'Subclinical primary hypothyroidism (SCH)',
       'Subclinical primary hyperthyroidism\n(SCT)', 'Low T3', 'Ipotiroidismo',
       'Ipertiroidismo', 'Angina', 'Previous CABG', 'Previous PCI',
       'Previous Myocardial Infarction', 'Acute Myocardial Infarction',
       'Angiography', 'CAD', 'Documented resting \nor exertional ischemia',
       'Post-ischemic Dilated\nCardiomyopathy',
       'Primary Dilated\nCardiomyopathy', 'Normal', 'SindromeX', 'AMI', 'PMI',
       'Aritmie', 'MIN', 'MIO', 'Miocardite', 'Pericardite', 'Endocardite',
       'Valvulopatie', 'MalattiaVasoAorta', 'Ipertensione', 'EmboliaPolmonare',
       'EPA', 'Smoke\nHistory of smoke', 'Diabetes\nHistory of diabetes',
       'Hypertension\nHistory of hypertension',
       'Dyslipidemia\nHystory of dyslipidemia', 'Atrial Fibrillation',
       'B-Blockers', 'Amiodarone', 'Calcium channel blockers', 'Diuretics',
       'Antiplatelet', 'Nitrates', 'ACE

# Questions 

Tre domande:
- I pazienti sono disgiunti? (potremmo controllare ma non facile perchè abbiamo variabili diverse)?
- Quanti valori nulli hanno le colonne nuove? vale la pena tenerli? (ad occhio si a prescindere)
- Come facciamo a trovare data di morte dei nuovi pazienti. E nel caso, quale task di classificazione possiamo fare per testare i dati sui nuovi pazienti
- Altre colonne comuni?
- Alcune colonne forse dovrebbero essere tolte? (in generale chiedere ogni colonna cos'e')
- Facciamo un test su questo nuovo senza riallenare? Dobbiamo mergiare e aumentare il numero totale di colonne? dobbiamo solo rifare tutto su questo? Miss Alina, Mr Priami siate chiari...